In [17]:
import pandas as pd
import os

In [18]:
raw_events_data = pd.read_csv("../datasets/raw_events_data.csv")
raw_labtest_data = pd.read_csv("../datasets/raw_labtest_data.csv")
raw_target_data = pd.read_csv("../datasets/raw_target_data.csv")

In [121]:
def generate_description(csv_files):
    all_descriptions = []
    total_entries = 0
    total_columns = 0
    example_data = {}
    
    for csv_file in csv_files:
        # Load the CSV file
        df = pd.read_csv(csv_file)
        
        # Extract information about the dataset
        num_entries = len(df)
        num_columns = len(df.columns)
        
        # Update total counts
        total_entries += num_entries
        total_columns += num_columns

        # Get detailed descriptions of each column
        column_descriptions = []
        for column in df.columns:
            col_info = f"{column}. "
            col_info += f"No missing values." if df[column].notnull().all() else f"Contains {df[column].isnull().sum()} missing values."
            col_info += f" Data type: {df[column].dtype}."
            column_descriptions.append(col_info)

        # Generate the description
        description = f"The dataset {csv_file} contains {num_entries} entries and {num_columns} columns. Here is a detailed description of each column:\n"
        description += "\n".join(column_descriptions)
        all_descriptions.append(description)
        
        # Get example data
        example_data[csv_file] = df.head(3)

    # Summary description
    summary_description = f"There are a total of {len(csv_files)} CSV files, containing {total_entries} entries and {total_columns} columns in total."
    
    descriptions = summary_description+'\n'.join(all_descriptions) + str(example_data)
    
    return descriptions

In [123]:
# List of CSV files
csv_files = ['raw_events_data.csv', 'raw_labtest_data.csv', 'raw_target_data.csv']
data_path = "../datasets" 
csv_files = [os.path.join(data_path,csv_file) for csv_file in csv_files]
# Generate descriptions for each CSV file
descriptions = generate_description(csv_files)

print(descriptions)

There are a total of 3 CSV files, containing 58018 entries and 12 columns in total.The dataset ../datasets/raw_events_data.csv contains 375 entries and 4 columns. Here is a detailed description of each column:
PatientID. No missing values. Data type: int64.
RecordTime. Contains 14 missing values. Data type: object.
Name. No missing values. Data type: object.
Value. No missing values. Data type: float64.
The dataset ../datasets/raw_labtest_data.csv contains 55939 entries and 4 columns. Here is a detailed description of each column:
PatientID. No missing values. Data type: int64.
RecordTime. Contains 14 missing values. Data type: object.
Name. No missing values. Data type: object.
Value. No missing values. Data type: float64.
The dataset ../datasets/raw_target_data.csv contains 1704 entries and 4 columns. Here is a detailed description of each column:
PatientID. No missing values. Data type: int64.
RecordTime. No missing values. Data type: object.
Outcome. No missing values. Data type: i

In [226]:
prompt = """作为一名负责制定医疗数据分析题目的出题专家，你的任务是根据提供的医疗数据分析概念和数据集情况，设计出难度设定分别为简单的五个题目、为中等的 5 个题目、为困难的5个问题,注意题目的逻辑性和常识性，同时需要以中文口语化的形式进行表达。这些问题请严格根据给定相关数据分析概念和数据集信息，不得出现超范围的自由发挥内容，并确定能使用 Python 进行编码解决，你不需要提供具体解答。其中简单、中等、困难的划定标准如下所示：易：基础问题，涉及单一概念或简单的数据处理任务。中：进阶问题，可能涉及多个概念的综合应用。难：高级问题，要求深入理解复杂的数据分析概念和方法。

请根据数据集描述：{description}以及相关数据分析概念：{concept}来构建问题。确保每个问题描述都很具体且明确，并格外清晰地标明了所涉及的分析概念和相应的数据集。
问题相关的举例为\n{example}

输出格式要求：请以 csv 格式输出，列名分别为：相关数据分析概念(固定为{key})，难度，问题内容，使用到的相关概念(具体说明)
"""

In [227]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import find_dotenv,load_dotenv
import openai
import os

In [228]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [229]:
problem_template = PromptTemplate(input_variables=["key","description","concept","example"], template=prompt)

In [230]:
model = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0, model_kwargs={"seed": 42})

In [231]:
concept_dict = {
  "数据查询": {
    "定义": "根据特定的条件和需求，从数据集中检索出符合要求的数据记录的过程。查询可以基于单一条件或多个条件进行，可以包括数值比较、文本匹配、日期范围等。",
    "举例":"有哪位病人的住院时间超过15天？"
  },
  "摘要统计": {
    "定义": "对数据集中的关键特征进行汇总和描述的过程，通常包括中心趋势（如平均值、中位数）、离散程度（如标准差、方差）、数据分布（如最小值、最大值、百分位数）、形状特征（如偏度、峰度）。这些统计量提供了对数据集整体特征的简明概括，有助于更好地理解数据的基本情况和趋势。",
    "举例":"计算raw_target_data.csv中患者住院时长的平均值"

  },
  "相关性分析": {
    "定义": "评估两个或多个变量之间关系的方法，通常通过相关系数来衡量变量之间的线性相关程度。",
    "举例":"请计算raw_events_data.csv中血压与raw_target_data.csv中住院时长之间的相关系数。"
  },
  "异常值检测": {
    "定义": "数据集的异常值检测是识别与大多数数据明显不同的观测值的过程，通常通过比较数据点与数据集的整体分布特征来进行。常用的方法包括基于统计量的方法（如Z-score）、基于距离的方法（如离群点检测算法）、基于密度的方法（如LOF算法）等。",
    "举例":"在raw_target_data.csv数据集中，使用Z-score方法检测患者住院时长的异常值。"
  },
  "数据预处理": {
    "定义": "医疗数据预处理是一个关键步骤，它涉及将原始医疗数据转换为适合进一步分析和模型训练的格式。预处理包括数据清洗、格式标准化、缺失值填充、异常值处理、特征提取、数据合并、统计分析、数据集划分以及归一化等操作。",
    "举例":"合并三个数据集，并根据患者编号和记录时间进行排序。"
  },
  "模型调用预测": {
    "定义": "医疗模型调用预测是一个将训练好的医疗模型应用于新数据以进行健康风险评估和疾病预测的过程。这一过程通常包括加载预训练的模型(Concare,Adacare,,GRU,LSTM)、准备输入数据（包括患者的历史医疗记录和实时监测数据）、执行模型推理以生成预测结果。",
    "举例":"请使用Concare模型对合并后的数据进行模型调用和预测"
  },
  "事后可解释性分析": {
    "定义": "事后可解释性分析是指在医疗模型完成预测后，对模型的决策过程进行分析，以理解模型是如何得出特定预测结果的。这一分析过程包括特征重要性评估、风险曲线绘制、AI建议生成以及数据维度降低等方法。",
    "举例":"请使用 PCA的方法对患者的数据进行降维分析"
  },
  "医疗专业绘图操作": {
    "定义": "医疗专业绘图操作是指使用特定的可视化技术将医疗数据和模型分析结果以图形的方式展现出来的过程。根据所提供的文件内容，这些操作包括数据集分布的直方图绘制、特征重要性的条形图展示、患者风险曲线的绘制以及患者嵌入的降维展示等。",
    "举例":"请使用Concare模型对合并后的数据进行模型调用和预测并绘制患者的风险曲线"
  }
}

In [232]:
concept_list = ["数据查询","摘要统计","相关性分析","异常值检测","数据预处理","模型调用预测","事后可解释性分析","医疗专业绘图操作"]

In [233]:
problem = {}

In [234]:
concept_dict["摘要统计"]

{'定义': '对数据集中的关键特征进行汇总和描述的过程，通常包括中心趋势（如平均值、中位数）、离散程度（如标准差、方差）、数据分布（如最小值、最大值、百分位数）、形状特征（如偏度、峰度）。这些统计量提供了对数据集整体特征的简明概括，有助于更好地理解数据的基本情况和趋势。',
 '举例': '计算raw_target_data.csv中患者住院时长的平均值'}

In [235]:
p =problem_template.partial(key=concept_list[4],description=descriptions,concept=concept_list[4]+":"+concept_dict[concept_list[4]]['定义'],example=concept_dict[concept_list[4]]['举例']).format()

In [236]:
p

"作为一名负责制定医疗数据分析题目的出题专家，你的任务是根据提供的医疗数据分析概念和数据集情况，设计出难度设定分别为简单的五个题目、为中等的 5 个题目、为困难的5个问题,注意题目的逻辑性和常识性，同时需要以中文口语化的形式进行表达。这些问题请严格根据给定相关数据分析概念和数据集信息，不得出现超范围的自由发挥内容，并确定能使用 Python 进行编码解决，你不需要提供具体解答。其中简单、中等、困难的划定标准如下所示：易：基础问题，涉及单一概念或简单的数据处理任务。中：进阶问题，可能涉及多个概念的综合应用。难：高级问题，要求深入理解复杂的数据分析概念和方法。\n\n请根据数据集描述：There are a total of 3 CSV files, containing 58018 entries and 12 columns in total.The dataset ../datasets/raw_events_data.csv contains 375 entries and 4 columns. Here is a detailed description of each column:\nPatientID. No missing values. Data type: int64.\nRecordTime. Contains 14 missing values. Data type: object.\nName. No missing values. Data type: object.\nValue. No missing values. Data type: float64.\nThe dataset ../datasets/raw_labtest_data.csv contains 55939 entries and 4 columns. Here is a detailed description of each column:\nPatientID. No missing values. Data type: int64.\nRecordTime. Contains 14 missing values. Data type: object.\nName. No missing values. Data type: object.\nValue. No

In [237]:
model.invoke(p)

AIMessage(content='```csv\n数据预处理,易,"请统计\'raw_events_data.csv\'中的患者总数。","数据统计"\n数据预处理,易,"找出\'raw_labtest_data.csv\'中，记录时间缺失的条目数。","缺失值处理"\n数据预处理,易,"计算\'raw_target_data.csv\'中，所有患者住院天数(LOS)的平均值。","数据统计"\n数据预处理,易,"将\'raw_events_data.csv\'中的记录时间格式从\'年/月/日\'转换为\'年-月-日\'。","格式标准化"\n数据预处理,易,"在\'raw_labtest_data.csv\'中，找出血红蛋白(hemoglobin)值最高的患者ID。","数据统计"\n数据预处理,中,"合并\'raw_events_data.csv\'和\'raw_labtest_data.csv\'，并去除所有含有缺失值的行。","数据合并,缺失值处理"\n数据预处理,中,"对\'raw_labtest_data.csv\'中的Value列进行归一化处理。","归一化"\n数据预处理,中,"在\'raw_events_data.csv\'中，将性别(Sex)的值从1和0转换为\'Male\'和\'Female\'。","数据清洗"\n数据预处理,中,"找出\'raw_target_data.csv\'和\'raw_labtest_data.csv\'中，共同出现的患者ID列表。","数据合并"\n数据预处理,中,"对\'raw_events_data.csv\'中的Value列，找出异常值并替换为该列的平均值。","异常值处理"\n数据预处理,难,"基于\'raw_target_data.csv\'中的Outcome列，将三个数据集分为两组（Outcome为0的一组，Outcome为1的另一组），并分别计算两组的平均住院天数(LOS)。","数据合并,统计分析"\n数据预处理,难,"创建一个新的数据集，包含\'raw_labtest_data.csv\'中每个患者的最后一次实验室检测结果。","特征提取"\n数据预处理,难,"对\'raw_events_data.csv\'和\'raw_labtest_data.csv\'进行合并，并填充\'raw_even

In [203]:
for concept in concept_list:
    chain = problem_template.partial(key=concept,description=descriptions,concept=concept_dict[concept]['定义'],example=concept_dict[concept]['举例']) | model
    output = chain.invoke({})
    problem[concept] = output.content

KeyboardInterrupt: 

In [147]:
problem

{'数据查询': '```csv\n相关数据分析概念,难度,问题内容,使用到的相关概念\n数据查询,易,"请找出所有男性病人的ID。",单一条件查询\n数据查询,易,"请找出所有住院时间（LOS）等于10天的病人记录。",单一条件查询\n数据查询,中,"请找出所有在2020年2月份进行了检测的病人ID。",日期范围查询\n数据查询,中,"请找出所有血红蛋白（hemoglobin）值超过120的病人记录。",数值比较查询\n数据查询,中,"请列出所有进行了白细胞计数（eosinophils(%)）检测的病人的检测时间和值。",文本匹配查询\n数据查询,难,"请找出所有男性病人中，血红蛋白值超过130的记录。",多条件综合查询\n数据查询,难,"请找出所有在2020年1月31日既进行了检测又有住院记录的病人ID。",多数据集综合查询\n数据查询,难,"请找出住院时间超过15天，且出院结果为0的病人记录。",多条件综合查询\n数据查询,难,"请找出所有检测了血小板计数（假设为platelet count）且值低于正常范围的病人记录。",数值比较查询和文本匹配查询\n数据查询,难,"请找出所有在2020年2月份有检测记录，且住院时间超过10天的病人ID。",日期范围查询和数值比较查询的综合应用\n```',
 '摘要统计': '```csv\n摘要统计,难度,问题内容,使用到的相关概念\n摘要统计,易,"计算raw_labtest_data.csv中所有患者的血红蛋白(hemoglobin)检测值的平均数是多少？","平均值"\n摘要统计,易,"找出raw_events_data.csv中，记录缺失时间(RecordTime)的患者ID。","数据清洗"\n摘要统计,易,"计算raw_target_data.csv中，所有患者住院时长(LOS)的中位数是多少？","中位数"\n摘要统计,易,"在raw_labtest_data.csv数据集中，计算eosinophils(%)检测的最大值和最小值。","最大值、最小值"\n摘要统计,易,"确定raw_events_data.csv中，性别(Sex)记录为1的患者数量。","数据计数"\n摘要统计,中,"在raw_labtest_data.csv中，对每种检测项(Name)，计算其值(Value)的标准差是多少？","标准差"

In [148]:
problem['医疗专业绘图操作']

'```csv\n相关数据分析概念,难度,问题内容,使用到的相关概念\n医疗专业绘图操作,易,"请使用Python绘制raw_labtest_data.csv中\'hemoglobin\'值的直方图，以观察其分布情况。",数据集分布的直方图绘制\n医疗专业绘图操作,易,"请绘制raw_events_data.csv中\'Sex\'字段的条形图，展示不同性别的患者数量。",特征重要性的条形图展示\n医疗专业绘图操作,中,"请结合raw_labtest_data.csv和raw_target_data.csv，绘制患者\'hemoglobin\'值与LOS之间的散点图，探索二者之间的关系。",患者风险曲线的绘制\n医疗专业绘图操作,中,"请对raw_labtest_data.csv中的所有检测项(Name字段)的平均值进行计算，并绘制条形图展示每个检测项的平均值。",特征重要性的条形图展示\n医疗专业绘图操作,困难,"请将raw_events_data.csv和raw_labtest_data.csv按PatientID和RecordTime合并，并使用PCA方法对合并后的数据进行降维，最后绘制降维后的数据分布图。",患者嵌入的降维展示\n医疗专业绘图操作,困难,"请使用raw_target_data.csv中的Outcome字段，绘制不同Outcome值的患者在raw_labtest_data.csv中\'hemoglobin\'值的分布直方图，以比较不同Outcome下\'hemoglobin\'值的分布差异。",数据集分布的直方图绘制\n医疗专业绘图操作,易,"请绘制raw_target_data.csv中\'LOS\'字段的直方图，以观察住院天数的分布情况。",数据集分布的直方图绘制\n医疗专业绘图操作,中,"请绘制raw_labtest_data.csv中\'Value\'字段随时间变化的趋势图，选择任一PatientID进行展示。",患者风险曲线的绘制\n医疗专业绘图操作,困难,"请结合raw_labtest_data.csv和raw_target_data.csv，使用逻辑回归模型预测Outcome，并绘制ROC曲线评估模型性能。",患者风险曲线的绘制\n医疗专业绘图操作,困难,"请对raw_events_data.csv、raw_labtest_data

In [149]:
import csv
for filename, content in problem.items():
    with open(f"{filename}.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # 写入数据
        for line in content.split("\n"):
            writer.writerow(line.split(","))

    print(f"CSV文件已保存到: {filename}.csv")

CSV文件已保存到: 数据查询.csv
CSV文件已保存到: 摘要统计.csv
CSV文件已保存到: 相关性分析.csv
CSV文件已保存到: 异常值检测.csv
CSV文件已保存到: 数据预处理.csv
CSV文件已保存到: 模型调用预测.csv
CSV文件已保存到: 事后可解释性分析.csv
CSV文件已保存到: 医疗专业绘图操作.csv


In [104]:
csv_file = "all_data.csv"

# 逐个保存字典中的每个键值对应的CSV文件内容到CSV文件
with open(csv_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # 写入表头
    writer.writerow(["文件名", "内容"])
    # 写入数据
    for filename, content in problem.items():
        writer.writerow([filename, content])

print(f"所有数据已保存到CSV文件: {csv_file}")

所有数据已保存到CSV文件: all_data.csv


In [42]:
import json
file_path = "problems.json"

# 将字典写入JSON文件
with open(file_path, "w", encoding="utf-8") as json_file:
    json.dump(problem, json_file, ensure_ascii=False)


In [41]:
print(problem['数据查询'])

### 简单问题

1. **查询特定患者的记录**  
   使用 `raw_events_data.csv` 数据集，找出所有 PatientID 为 12345 的记录。  
   涉及概念：基础查询。

2. **计算特定检测的平均值**  
   使用 `raw_labtest_data.csv` 数据集，计算 Name 为 "Glucose" 的所有记录的 Value 平均值。  
   涉及概念：基础查询与数值计算。

3. **列出所有有缺失 RecordTime 的患者ID**  
   使用 `raw_events_data.csv` 数据集，列出所有 RecordTime 为缺失值的 PatientID。  
   涉及概念：缺失值处理。

4. **找出特定日期内的记录**  
   使用 `raw_labtest_data.csv` 数据集，找出 RecordTime 在 "2023-01-01" 到 "2023-01-31" 之间的所有记录。  
   涉及概念：日期范围查询。

5. **计算特定患者的记录数量**  
   使用 `raw_target_data.csv` 数据集，计算 PatientID 为 67890 的记录数量。  
   涉及概念：基础查询。

### 中等问题

1. **查询特定检测值超过阈值的患者ID**  
   使用 `raw_labtest_data.csv` 数据集，找出所有 Name 为 "Hemoglobin" 且 Value 大于 13.5 的 PatientID。  
   涉及概念：条件查询与数值比较。

2. **合并患者记录和检测结果**  
   使用 `raw_events_data.csv` 和 `raw_labtest_data.csv` 数据集，根据 PatientID 合并两个数据集的记录。  
   涉及概念：数据合并。

3. **计算每个患者的检测记录数量**  
   使用 `raw_labtest_data.csv` 数据集，计算每个 PatientID 的记录数量，并列出前 10 个患者及其记录数量。  
   涉及概念：分组与聚合。

4. **找出具有特定 Outcome 的患者的平均 LOS**  
   使用 `raw_target_data.csv`